##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# MLMD Model Card Toolkit Demo

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/responsible_ai/model_card_toolkit/examples/MLMD_Model_Card_Toolkit_Demo"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/model-card-toolkit/blob/master/model_card_toolkit/documentation/examples/MLMD_Model_Card_Toolkit_Demo.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/model-card-toolkit/blob/master/model_card_toolkit/documentation/examples/MLMD_Model_Card_Toolkit_Demo.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/model-card-toolkit/model_card_toolkit/documentation/examples/MLMD_Model_Card_Toolkit_Demo.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

## Background

This notebook demonstrates how to generate a model card using the Model Card Toolkit with MLMD and TFX pipeline in a Jupyter/Colab environment. You can learn more about model cards at https://modelcards.withgoogle.com/about. 



## Setup
We first need to a) install and import the necessary packages, and b) download the data.

### Upgrade to Pip 20.2 and Install TFX

In [ ]:
!pip install --upgrade pip==20.2
!pip install "tfx==1.2.0"
!pip install model-card-toolkit

### Did you restart the runtime?

If you are using Google Colab, the first time that you run the cell above, you must restart the runtime (Runtime > Restart runtime ...). This is because of the way that Colab loads packages.

### Import packages

We import necessary packages, including standard TFX component classes and check the library versions.


In [ ]:
import os
import pprint
import tempfile
import urllib

import absl
import tensorflow as tf
import tensorflow_model_analysis as tfma
tf.get_logger().propagate = False
pp = pprint.PrettyPrinter()

import tfx
from tfx.components import CsvExampleGen
from tfx.components import Evaluator
from tfx.components import Pusher
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Trainer
from tfx.components import Transform
from tfx.components.base import executor_spec
from tfx.components.trainer.executor import GenericExecutor
from tfx.dsl.experimental import latest_blessed_model_resolver
from tfx.orchestration import metadata
from tfx.orchestration import pipeline
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.types import Channel
from tfx.types.standard_artifacts import Model
from tfx.types.standard_artifacts import ModelBlessing

import ml_metadata as mlmd

In [ ]:
print('TensorFlow version: {}'.format(tf.__version__))
print('TFX version: {}'.format(tfx.version.__version__))
print('MLMD version: {}'.format(mlmd.__version__))

### Set up pipeline paths

In [ ]:
# This is the root directory for your TFX pip package installation.
_tfx_root = tfx.__path__

# Set up logging.
absl.logging.set_verbosity(absl.logging.INFO)

### Download example data
We download the example dataset for use in our TFX pipeline.


In [ ]:
DATA_PATH = 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/' \
   'adult.data'
_data_root = tempfile.mkdtemp(prefix='tfx-data')
_data_filepath = os.path.join(_data_root, "data.csv")
urllib.request.urlretrieve(DATA_PATH, _data_filepath)

columns = [
  "Age", "Workclass", "fnlwgt", "Education", "Education-Num", "Marital-Status",
  "Occupation", "Relationship", "Race", "Sex", "Capital-Gain", "Capital-Loss",
  "Hours-per-week", "Country", "Over-50K"]

with open(_data_filepath, 'r') as f:
  content = f.read()
  content = content.replace(", <=50K", ', 0').replace(", >50K", ', 1')

with open(_data_filepath, 'w') as f:
  f.write(','.join(columns) + '\n' + content)

Take a quick look at the CSV file.

In [ ]:
!head {_data_filepath}

### Create the InteractiveContext
Last, we create an InteractiveContext, which will allow us to run TFX components interactively in this notebook.

In [ ]:
# Here, we create an InteractiveContext using default parameters. This will
# use a temporary directory with an ephemeral ML Metadata database instance.
# To use your own pipeline root or database, the optional properties
# `pipeline_root` and `metadata_connection_config` may be passed to
# InteractiveContext. Calls to InteractiveContext are no-ops outside of the
# notebook.
context = InteractiveContext(pipeline_name="Census Income Classification Pipeline")

## Run TFX components interactively
In the cells that follow, we create TFX components one-by-one, run each of them, and visualize their output artifacts. In this notebook, we won’t provide detailed explanations of each TFX component, but you can see what each does at [TFX Colab workshop](https://github.com/tensorflow/workshops/blob/master/tfx_labs/Lab_1_Pipeline_in_Colab.ipynb).

### ExampleGen

Create the `ExampleGen` component to split data into training and evaluation sets, convert the data into `tf.Example` format, and copy data into the `_tfx_root` directory for other components to access. 

In [ ]:
example_gen = CsvExampleGen(input_base=_data_root)
context.run(example_gen)

In [ ]:
artifact = example_gen.outputs['examples'].get()[0]
print(artifact.split_names, artifact.uri)

Let’s take a look at the first three training examples:

In [ ]:
# Get the URI of the output artifact representing the training examples, which is a directory
train_uri = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 3 records and decode them.
for tfrecord in dataset.take(3):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  pp.pprint(example)

### StatisticsGen

`StatisticsGen` takes as input the dataset we just ingested using `ExampleGen` and allows you to perform some analysis of your dataset using TensorFlow Data Validation (TFDV).

In [ ]:
statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples'])
context.run(statistics_gen)

After `StatisticsGen` finishes running, we can visualize the outputted statistics. Try playing with the different plots!

In [ ]:
context.show(statistics_gen.outputs['statistics'])

### SchemaGen

`SchemaGen` will take as input the statistics that we generated with `StatisticsGen`, looking at the training split by default.

In [ ]:
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=False)
context.run(schema_gen)

In [ ]:
context.show(schema_gen.outputs['schema'])

To learn more about schemas, see [the SchemaGen documentation](https://www.tensorflow.org/tfx/guide/schemagen).

### Transform

`Transform` will take as input the data from `ExampleGen`, the schema from `SchemaGen`, as well as a module that contains user-defined Transform code.

Let's see an example of user-defined Transform code below (for an introduction to the TensorFlow Transform APIs, [see the tutorial](https://www.tensorflow.org/tfx/tutorials/transform/simple)).



In [ ]:
_census_income_constants_module_file = 'census_income_constants.py'

In [ ]:
%%writefile {_census_income_constants_module_file}

# Categorical features are assumed to each have a maximum value in the dataset.
MAX_CATEGORICAL_FEATURE_VALUES = [20]

CATEGORICAL_FEATURE_KEYS = ["Education-Num"]


DENSE_FLOAT_FEATURE_KEYS = ["Capital-Gain", "Hours-per-week", "Capital-Loss"]

# Number of buckets used by tf.transform for encoding each feature.
FEATURE_BUCKET_COUNT = 10

BUCKET_FEATURE_KEYS = ["Age"]

# Number of vocabulary terms used for encoding VOCAB_FEATURES by tf.transform
VOCAB_SIZE = 200

# Count of out-of-vocab buckets in which unrecognized VOCAB_FEATURES are hashed.
OOV_SIZE = 10

VOCAB_FEATURE_KEYS = ["Workclass", "Education", "Marital-Status", "Occupation", 
                      "Relationship", "Race", "Sex", "Country"]

# Keys
LABEL_KEY = "Over-50K"

def transformed_name(key):
  return key + '_xf'

In [ ]:
_census_income_transform_module_file = 'census_income_transform.py'

In [ ]:
%%writefile {_census_income_transform_module_file}

import tensorflow as tf
import tensorflow_transform as tft

import census_income_constants

_DENSE_FLOAT_FEATURE_KEYS = census_income_constants.DENSE_FLOAT_FEATURE_KEYS
_VOCAB_FEATURE_KEYS = census_income_constants.VOCAB_FEATURE_KEYS
_VOCAB_SIZE = census_income_constants.VOCAB_SIZE
_OOV_SIZE = census_income_constants.OOV_SIZE
_FEATURE_BUCKET_COUNT = census_income_constants.FEATURE_BUCKET_COUNT
_BUCKET_FEATURE_KEYS = census_income_constants.BUCKET_FEATURE_KEYS
_CATEGORICAL_FEATURE_KEYS = census_income_constants.CATEGORICAL_FEATURE_KEYS
_LABEL_KEY = census_income_constants.LABEL_KEY
_transformed_name = census_income_constants.transformed_name


def preprocessing_fn(inputs):
  """tf.transform's callback function for preprocessing inputs.
  Args:
    inputs: map from feature keys to raw not-yet-transformed features.
  Returns:
    Map from string feature key to transformed feature operations.
  """
  outputs = {}
  for key in _DENSE_FLOAT_FEATURE_KEYS:
    # Preserve this feature as a dense float, setting nan's to the mean.
    outputs[_transformed_name(key)] = tft.scale_to_z_score(
        _fill_in_missing(inputs[key]))

  for key in _VOCAB_FEATURE_KEYS:
    # Build a vocabulary for this feature.
    outputs[_transformed_name(key)] = tft.compute_and_apply_vocabulary(
        _fill_in_missing(inputs[key]),
        top_k=_VOCAB_SIZE,
        num_oov_buckets=_OOV_SIZE)

  for key in _BUCKET_FEATURE_KEYS:
    outputs[_transformed_name(key)] = tft.bucketize(
        _fill_in_missing(inputs[key]), _FEATURE_BUCKET_COUNT)

  for key in _CATEGORICAL_FEATURE_KEYS:
    outputs[_transformed_name(key)] = _fill_in_missing(inputs[key])

  label = _fill_in_missing(inputs[_LABEL_KEY])
  outputs[_transformed_name(_LABEL_KEY)] = label
  
  return outputs


def _fill_in_missing(x):
  """Replace missing values in a SparseTensor.
  Fills in missing values of `x` with '' or 0, and converts to a dense tensor.
  Args:
    x: A `SparseTensor` of rank 2.  Its dense shape should have size at most 1
      in the second dimension.
  Returns:
    A rank 1 tensor where missing values of `x` have been filled in.
  """
  default_value = '' if x.dtype == tf.string else 0
  return tf.squeeze(
      tf.sparse.to_dense(
          tf.SparseTensor(x.indices, x.values, [x.dense_shape[0], 1]),
          default_value),
      axis=1)

In [ ]:
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath(_census_income_transform_module_file))
context.run(transform)

In [ ]:
transform.outputs

### Trainer
Let's see an example of user-defined model code below (for an introduction to the TensorFlow Keras APIs, [see the tutorial](https://www.tensorflow.org/guide/keras)):

In [ ]:
_census_income_trainer_module_file = 'census_income_trainer.py'

In [ ]:
%%writefile {_census_income_trainer_module_file}

from typing import List, Text

import os
import absl
import datetime
import tensorflow as tf
import tensorflow_transform as tft

from tfx.components.trainer.executor import TrainerFnArgs

import census_income_constants

_DENSE_FLOAT_FEATURE_KEYS = census_income_constants.DENSE_FLOAT_FEATURE_KEYS
_VOCAB_FEATURE_KEYS = census_income_constants.VOCAB_FEATURE_KEYS
_VOCAB_SIZE = census_income_constants.VOCAB_SIZE
_OOV_SIZE = census_income_constants.OOV_SIZE
_FEATURE_BUCKET_COUNT = census_income_constants.FEATURE_BUCKET_COUNT
_BUCKET_FEATURE_KEYS = census_income_constants.BUCKET_FEATURE_KEYS
_CATEGORICAL_FEATURE_KEYS = census_income_constants.CATEGORICAL_FEATURE_KEYS
_MAX_CATEGORICAL_FEATURE_VALUES = census_income_constants.MAX_CATEGORICAL_FEATURE_VALUES
_LABEL_KEY = census_income_constants.LABEL_KEY
_transformed_name = census_income_constants.transformed_name


def _transformed_names(keys):
  return [_transformed_name(key) for key in keys]


def _gzip_reader_fn(filenames):
  """Small utility returning a record reader that can read gzip'ed files."""
  return tf.data.TFRecordDataset(
      filenames,
      compression_type='GZIP')


def _get_serve_tf_examples_fn(model, tf_transform_output):
  """Returns a function that parses a serialized tf.Example and applies TFT."""

  model.tft_layer = tf_transform_output.transform_features_layer()

  @tf.function
  def serve_tf_examples_fn(serialized_tf_examples):
    """Returns the output to be used in the serving signature."""
    feature_spec = tf_transform_output.raw_feature_spec()
    feature_spec.pop(_LABEL_KEY)
    parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)

    transformed_features = model.tft_layer(parsed_features)
    if _transformed_name(_LABEL_KEY) in transformed_features:
      transformed_features.pop(_transformed_name(_LABEL_KEY))

    return model(transformed_features)

  return serve_tf_examples_fn


def _input_fn(file_pattern: List[Text],
              tf_transform_output: tft.TFTransformOutput,
              batch_size: int = 200) -> tf.data.Dataset:
  """Generates features and label for tuning/training.

  Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    tf_transform_output: A TFTransformOutput.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

  Returns:
    A dataset that contains (features, indices) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of label indices.
  """
  transformed_feature_spec = (
      tf_transform_output.transformed_feature_spec().copy())

  dataset = tf.data.experimental.make_batched_features_dataset(
      file_pattern=file_pattern,
      batch_size=batch_size,
      features=transformed_feature_spec,
      reader=_gzip_reader_fn,
      label_key=_transformed_name(_LABEL_KEY))

  return dataset


def _build_keras_model(hidden_units: List[int] = None) -> tf.keras.Model:
  """Creates a DNN Keras model.

  Args:
    hidden_units: [int], the layer sizes of the DNN (input layer first).

  Returns:
    A keras Model.
  """
  real_valued_columns = [
      tf.feature_column.numeric_column(key, shape=())
      for key in _transformed_names(_DENSE_FLOAT_FEATURE_KEYS)
  ]
  categorical_columns = [
      tf.feature_column.categorical_column_with_identity(
          key, num_buckets=_VOCAB_SIZE + _OOV_SIZE, default_value=0)
      for key in _transformed_names(_VOCAB_FEATURE_KEYS)
  ]
  categorical_columns += [
      tf.feature_column.categorical_column_with_identity(
          key, num_buckets=_FEATURE_BUCKET_COUNT, default_value=0)
      for key in _transformed_names(_BUCKET_FEATURE_KEYS)
  ]
  categorical_columns += [
      tf.feature_column.categorical_column_with_identity(  # pylint: disable=g-complex-comprehension
          key,
          num_buckets=num_buckets,
          default_value=0) for key, num_buckets in zip(
              _transformed_names(_CATEGORICAL_FEATURE_KEYS),
              _MAX_CATEGORICAL_FEATURE_VALUES)
  ]
  indicator_column = [
      tf.feature_column.indicator_column(categorical_column)
      for categorical_column in categorical_columns
  ]

  model = _wide_and_deep_classifier(
      # TODO(b/139668410) replace with premade wide_and_deep keras model
      wide_columns=indicator_column,
      deep_columns=real_valued_columns,
      dnn_hidden_units=hidden_units or [100, 70, 50, 25])
  return model


def _wide_and_deep_classifier(wide_columns, deep_columns, dnn_hidden_units):
  """Build a simple keras wide and deep model.

  Args:
    wide_columns: Feature columns wrapped in indicator_column for wide (linear)
      part of the model.
    deep_columns: Feature columns for deep part of the model.
    dnn_hidden_units: [int], the layer sizes of the hidden DNN.

  Returns:
    A Wide and Deep Keras model
  """
  # Following values are hard coded for simplicity in this example,
  # However prefarably they should be passsed in as hparams.

  # Keras needs the feature definitions at compile time.
  # TODO(b/139081439): Automate generation of input layers from FeatureColumn.
  input_layers = {
      colname: tf.keras.layers.Input(name=colname, shape=(), dtype=tf.float32)
      for colname in _transformed_names(_DENSE_FLOAT_FEATURE_KEYS)
  }
  input_layers.update({
      colname: tf.keras.layers.Input(name=colname, shape=(), dtype='int32')
      for colname in _transformed_names(_VOCAB_FEATURE_KEYS)
  })
  input_layers.update({
      colname: tf.keras.layers.Input(name=colname, shape=(), dtype='int32')
      for colname in _transformed_names(_BUCKET_FEATURE_KEYS)
  })
  input_layers.update({
      colname: tf.keras.layers.Input(name=colname, shape=(), dtype='int32')
      for colname in _transformed_names(_CATEGORICAL_FEATURE_KEYS)
  })

  # TODO(b/161816639): SparseFeatures for feature columns + Keras.
  deep = tf.keras.layers.DenseFeatures(deep_columns)(input_layers)
  for numnodes in dnn_hidden_units:
    deep = tf.keras.layers.Dense(numnodes)(deep)
  wide = tf.keras.layers.DenseFeatures(wide_columns)(input_layers)

  output = tf.keras.layers.Dense(
      1, activation='sigmoid')(
          tf.keras.layers.concatenate([deep, wide]))

  model = tf.keras.Model(input_layers, output)
  model.compile(
      loss='binary_crossentropy',
      optimizer=tf.keras.optimizers.Adam(lr=0.001),
      metrics=[tf.keras.metrics.BinaryAccuracy()])
  model.summary(print_fn=absl.logging.info)
  return model


# TFX Trainer will call this function.
def run_fn(fn_args: TrainerFnArgs):
  """Train the model based on given args.

  Args:
    fn_args: Holds args used to train the model as name/value pairs.
  """
  # Number of nodes in the first layer of the DNN
  first_dnn_layer_size = 100
  num_dnn_layers = 4
  dnn_decay_factor = 0.7

  tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)

  train_dataset = _input_fn(fn_args.train_files, tf_transform_output, 40)
  eval_dataset = _input_fn(fn_args.eval_files, tf_transform_output, 40)

  model = _build_keras_model(
      # Construct layers sizes with exponetial decay
      hidden_units=[
          max(2, int(first_dnn_layer_size * dnn_decay_factor**i))
          for i in range(num_dnn_layers)
      ])

  # This log path might change in the future.
  log_dir = os.path.join(os.path.dirname(fn_args.serving_model_dir), 'logs')
  tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir=log_dir, update_freq='batch')
  model.fit(
      train_dataset,
      steps_per_epoch=fn_args.train_steps,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps,
      callbacks=[tensorboard_callback])

  signatures = {
      'serving_default':
          _get_serve_tf_examples_fn(model,
                                    tf_transform_output).get_concrete_function(
                                        tf.TensorSpec(
                                            shape=[None],
                                            dtype=tf.string,
                                            name='examples')),
  }
  model.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)

In [ ]:
trainer = Trainer(
    module_file=os.path.abspath(_census_income_trainer_module_file),
    custom_executor_spec=executor_spec.ExecutorClassSpec(GenericExecutor),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(num_steps=100),
    eval_args=trainer_pb2.EvalArgs(num_steps=50))
context.run(trainer)

### Evaluator
The `Evaluator` component computes model performance metrics over the evaluation set. It uses the [TensorFlow Model Analysis](https://www.tensorflow.org/tfx/model_analysis/get_started) library. 

`Evaluator` will take as input the data from `ExampleGen`, the trained model from `Trainer`, and slicing configuration. The slicing configuration allows you to slice your metrics on feature values. See an example of this configuration below:

In [ ]:
from google.protobuf.wrappers_pb2 import BoolValue

eval_config = tfma.EvalConfig(
    model_specs=[
        # This assumes a serving model with signature 'serving_default'. If
        # using estimator based EvalSavedModel, add signature_name: 'eval' and 
        # remove the label_key.
        tfma.ModelSpec(label_key="Over-50K")
    ],
    metrics_specs=[
        tfma.MetricsSpec(
            # The metrics added here are in addition to those saved with the
            # model (assuming either a keras model or EvalSavedModel is used).
            # Any metrics added into the saved model (for example using
            # model.compile(..., metrics=[...]), etc) will be computed
            # automatically.
            # To add validation thresholds for metrics saved with the model,
            # add them keyed by metric name to the thresholds map.
            metrics=[
                tfma.MetricConfig(class_name='ExampleCount'),
                tfma.MetricConfig(class_name='BinaryAccuracy'),
                tfma.MetricConfig(class_name='FairnessIndicators',
                                  config='{ "thresholds": [0.5] }'),
            ]
        )
    ],
    slicing_specs=[
        # An empty slice spec means the overall slice, i.e. the whole dataset.
        tfma.SlicingSpec(),
        # Data can be sliced along a feature column. In this case, data is
        # sliced by feature column Race and Sex.
        tfma.SlicingSpec(feature_keys=['Race']),
        tfma.SlicingSpec(feature_keys=['Sex']),
        tfma.SlicingSpec(feature_keys=['Race', 'Sex']),
    ],
    options = tfma.Options(compute_confidence_intervals=BoolValue(value=True))
)

In [ ]:
# Use TFMA to compute a evaluation statistics over features of a model and
# validate them against a baseline.
evaluator = Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    eval_config=eval_config)
context.run(evaluator)

In [ ]:
evaluator.outputs

Using the `evaluation` output we can show the default visualization of global metrics on the entire evaluation set.

In [ ]:
context.show(evaluator.outputs['evaluation'])

## Populate Properties from ModelCard with Model Card Toolkit
Now that we’ve set up our TFX pipeline, we will use the Model Card Toolkit to extract key artifacts from the run and populate a Model Card.


### Connect to the MLMD store used by the InteractiveContext

In [ ]:
from ml_metadata.metadata_store import metadata_store
from IPython import display

mlmd_store = metadata_store.MetadataStore(context.metadata_connection_config)
model_uri = trainer.outputs["model"].get()[0].uri

### Use Model Card Toolkit

#### Initialize the Model Card Toolkit.

In [ ]:
import model_card_toolkit as mctlib

mct = mctlib.ModelCardToolkit(mlmd_store=mlmd_store, model_uri=model_uri)

#### Create Model Card workspace.

In [ ]:
model_card = mct.scaffold_assets()

#### Annotate more information into Model Card.

It is also important to document model information that might be important to downstream users, such as its limitations, intended use cases, trade offs, and ethical considerations. For each of these sections, we can directly add new JSON objects to represent this information. 


In [ ]:
model_card.model_details.name = 'Census Income Classifier'
model_card.model_details.overview = (
    'This is a wide and deep Keras model which aims to classify whether or not '
    'an individual has an income of over $50,000 based on various demographic '
    'features. The model is trained on the UCI Census Income Dataset. This is '
    'not a production model, and this dataset has traditionally only been used '
    'for research purposes. In this Model Card, you can review quantitative '
    'components of the model’s performance and data, as well as information '
    'about the model’s intended uses, limitations, and ethical considerations.'
)
model_card.model_details.owners = [
                                   mctlib.Owner(name='Model Cards Team',
                                         contact='model-cards@google.com')
]

In [ ]:
model_card.considerations.use_cases = [mctlib.UseCase(description=
    'This dataset that this model was trained on was originally created to '
    'support the machine learning community in conducting empirical analysis '
    'of ML algorithms. The Adult Data Set can be used in fairness-related '
    'studies that compare inequalities across sex and race, based on '
    'people’s annual incomes.')
]
model_card.considerations.limitations = [mctlib.Limitation(description=
    'This is a class-imbalanced dataset across a variety of sensitive classes.'
    ' The ratio of male-to-female examples is about 2:1 and there are far more'
    ' examples with the “white” attribute than every other race combined. '
    'Furthermore, the ratio of $50,000 or less earners to $50,000 or more '
    'earners is just over 3:1. Due to the imbalance across income levels, we '
    'can see that our true negative rate seems quite high, while our true '
    'positive rate seems quite low. This is true to an even greater degree '
    'when we only look at the “female” sub-group, because there are even '
    'fewer female examples in the $50,000+ earner group, causing our model to '
    'overfit these examples. To avoid this, we can try various remediation '
    'strategies in future iterations (e.g. undersampling, hyperparameter '
    'tuning, etc), but we may not be able to fix all of the fairness issues.')
]
model_card.considerations.ethical_considerations = [mctlib.Risk(
    name= 'We risk expressing the viewpoint that the attributes in this dataset '
          'are the only ones that are predictive of someone’s income, even '
          'though we know this is not the case.',
    mitigation_strategy= 'As mentioned, some interventions may need to be '
                         'performed to address the class imbalances in the dataset.'
        )
]

#### Filter and Add Graphs.

We can filter the graphs generated by the TFX components to include those most relevant for the Model Card using the function defined below. In this example, we filter for `race` and `sex`, two potentially sensitive attributes. 

Each Model Card will have up to three sections for graphs -- training dataset statistics, evaluation dataset statistics, and quantitative analysis of our model’s performance.



In [ ]:
# These are the graphs that will appear in the Quantiative Analysis portion of 
# the Model Card. Feel free to add or remove from this list. 
TARGET_EVAL_GRAPH_NAMES = [
  'fairness_indicators_metrics/false_positive_rate@0.5',
  'fairness_indicators_metrics/false_negative_rate@0.5',
  'binary_accuracy',
  'example_count | Race_X_Sex',
]

# These are the graphs that will appear in both the Train Set and Eval Set 
# portions of the Model Card. Feel free to add or remove from this list. 
TARGET_DATASET_GRAPH_NAMES = [
  'counts | Race',
  'counts | Sex',
]

def filter_graphs(graphics, target_graph_names):
  result = []
  for graph in graphics:
    for target_graph_name in target_graph_names:
      if graph.name.startswith(target_graph_name):
        result.append(graph)
  result.sort(key=lambda g: g.name)
  return result

# Populating the three different sections using the filter defined above. To 
# see all the graphs available in a section, we can iterate through each of the
# different collections. 
model_card.quantitative_analysis.graphics.collection = filter_graphs(
    model_card.quantitative_analysis.graphics.collection, TARGET_EVAL_GRAPH_NAMES)
model_card.model_parameters.data[0].graphics.collection = filter_graphs(
   model_card.model_parameters.data[0].graphics.collection, TARGET_DATASET_GRAPH_NAMES)
model_card.model_parameters.data[1].graphics.collection = filter_graphs(
    model_card.model_parameters.data[1].graphics.collection, TARGET_DATASET_GRAPH_NAMES)

We then add (optional) descriptions for each of the each of the graph sections. 

In [ ]:
model_card.model_parameters.data[0].name = 'train_set'
model_card.model_parameters.data[0].graphics.description = (
    'This section includes graphs displaying the class distribution for the '
    '“Race” and “Sex” attributes in our training dataset. We chose to '
    'show these graphs in particular because we felt it was important that '
    'users see the class imbalance.'
)
model_card.model_parameters.data[1].name = 'eval_set'
model_card.model_parameters.data[1].graphics.description = (
    'Like the training set, we provide graphs showing the class distribution '
    'of the data we used to evaluate our model’s performance. '
)
model_card.quantitative_analysis.graphics.description = (
    'These graphs show how the model performs for data sliced by “Race”, '
    '“Sex” and the intersection of these attributes. The metrics we chose '
    'to display are “Accuracy”, “False Positive Rate”, and “False '
    'Negative Rate”, because we anticipated that the class imbalances might '
    'cause our model to underperform for certain groups.'
)

In [ ]:
mct.update_model_card_json(model_card)

#### Generate the Model Card.
We can now display the Model Card in HTML format.


In [ ]:
html = mct.export_format()

In [ ]:
display.display(display.HTML(html))